In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Dict, Optional

c:\PythonProject\potenup_10\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dolmary_schedule_toolkit import DolmaryScheduleToolkit

PydanticUserError: Field 'description' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.12/u/model-field-overridden

In [ ]:
toolkits = DolmaryScheduleToolkit()
tools = toolkits.get_tools()
tools

In [ ]:
model = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0
)

prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 돌마리 스케쥴을 관리하는 AI 비서야. 주어지는 도구를 잘 활용해서 사용자의 요청에 답하도록 해"),
        ("placeholder", "{history}"),
        ("human", "{question}"),
        ("placeholder", "{agent_scratchpad}")
    ])

agent = create_openai_tools_agent(
    llm = model, 
    tools = tools,
    prompt = prompt,
)

executor = AgentExecutor(
    agent = agent,
    tools = tools, 
    verbose= True
)

In [ ]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

stores: Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id:str):
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

agent_history = RunnableWithMessageHistory(
    executor,
    lambda session_id : get_store(session_id),
    input_messages_key= "question",
    history_messages_key="history"
)

In [ ]:
cfg = {"configurable":{"session_id":"user-123"}}
agent_history.invoke({
    "question" : "오늘 돌마리 운동회 12시 30분에 파워댄스 일정 등록해줘"
},config = cfg)

In [ ]:
agent_history.invoke({
    "question" : "돌마리 스케쥴 전체 보여줘"
},config = cfg)